In [1]:
pip install transformers datasets torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel

# Load data from Sheet 1 in the Excel file (replace 'url' with the actual file path)
url = 'Book.xlsx'
sheet1_data = pd.read_excel(url, sheet_name='Sheet1')  # Load specifically 'Sheet1'

# Load pre-trained RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Function to compute RoBERTa embeddings for text
def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    # Average token embeddings for simplicity
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy().flatten()

# Extract RoBERTa embeddings for the 'Input' column from Sheet 1
roberta_input_embeddings = sheet1_data['Input'].apply(lambda text: get_roberta_embedding(str(text)))

# Create a DataFrame to store the embeddings
# Convert the list of embeddings into a DataFrame and name the columns
embeddings_df = pd.DataFrame(roberta_input_embeddings.tolist(), columns=[f'embed_{i}' for i in range(roberta_input_embeddings.iloc[0].size)])

# Add the 'Output' column from the original DataFrame
embeddings_df['output'] = sheet1_data['Output']

# Export embeddings to Excel
output_file = 'roberta_embeddings_input_output_custom_columns.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    embeddings_df.to_excel(writer, sheet_name='RoBERTa Embeddings', index=False)

print(f'RoBERTa embeddings saved to {output_file}')


C:\Users\tarun\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTa embeddings saved to roberta_embeddings_input_output_custom_columns.xlsx
